<a href="https://colab.research.google.com/github/Ale-Coeto/vision/blob/main/naos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path


In [ ]:
images_dir = "raw"
results_dir = "output"
bgs = "backgrounds"
dataset = "dataset"

if not os.path.exists(images_dir):
  os.makedirs(images_dir)

if not os.path.exists(results_dir):
  os.makedirs(results_dir)

if not os.path.exists("masks"):
  os.makedirs("masks")

if not os.path.exists(bgs):
  os.makedirs(bgs)

if not os.path.exists("dataset"):
  os.makedirs("dataset")

imgs_num = 37



In [ ]:
!pip install segment_anything

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

In [ ]:
sam_model = "h"

In [ ]:
if sam_model =="h":
  sam_checkpoint = "sam_vit_h_4b8939.pth"
  model_type = "vit_h"
else:
  sam_checkpoint = "sam_vit_l_0b3195.pth"
  model_type = "vit_l"

In [ ]:
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [ ]:
device = "cuda"

In [ ]:
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

RuntimeError: ignored

In [ ]:
images = [str("NAO" + str(i) + ".jpeg") for i in range(1, 38)] #38


In [ ]:
!pip install mediapipe

In [ ]:
import mediapipe as mp

In [ ]:
# Calling the pose solution from MediaPipe
mp_pose = mp.solutions.pose

# Calling the solution for image drawing from MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
!ls

masks  output  raw  sample_data  sam_vit_h_4b8939.pth


In [ ]:
i = 1
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
    for file in images:
        print(file)
        image = cv2.imread(os.path.join(images_dir, file))
        raw = image.copy()
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        x_min = 10000
        x_max = 0
        y_min = 10000
        y_max = 0
        margin_x = 290
        margin_y = 240
        if not results.pose_landmarks:
            continue
        for landmark in results.pose_landmarks.landmark:
            x_min = min(x_min, landmark.x)
            x_max = max(x_max, landmark.x)
            y_min = min(y_min, landmark.y)
            y_max = max(y_max, landmark.y)

        x_min = int(x_min * image.shape[1])
        x_max = int(x_max * image.shape[1])
        y_min = int(y_min * image.shape[0])
        y_max = int(y_max * image.shape[0])

        x_min = max(0,x_min - margin_x)
        y_min = max(0,y_min - margin_y)
        x_max = min(image.shape[1],x_max + margin_x)
        y_max = min(image.shape[0],y_max + margin_y)

        bbox = np.array([x_min, y_min, x_max, y_max])
        xywh = np.array([x_min, y_min, x_max - x_min, y_max - y_min])

        predictor.set_image(image)

        mask, _, _ = predictor.predict(
          point_coords=None,
          point_labels=None,
          box=bbox,
          multimask_output=False,
        )


        contours, _ = cv2.findContours(mask[0].astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Your call to find the contours
        # threshold input image using otsu thresholding as mask and refine with morphology
        ret, pngmask = cv2.threshold(mask[0].astype(np.uint8), 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        kernel = np.ones((9,9), np.uint8)
        pngmask = cv2.morphologyEx(pngmask, cv2.MORPH_CLOSE, kernel)
        pngmask = cv2.morphologyEx(pngmask, cv2.MORPH_OPEN, kernel)

        # put mask into alpha channel of result
        result = image.copy()
        result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
        result[:, :, 3] = pngmask

        #crop img
        x,y,w,h = cv2.boundingRect(contours[0])
        result = result[y:y+h, x:x+w]

        cv2.imwrite(f"{results_dir}/NAO{i}.png",result)
        i=i+1

        images=[]
annotations=[]
annotations2=[]

img_id=1
anno_id=int(0)

rescaling_min = 0.2
rescaling_max = 0.6

# Ratios at which these values will be modified
brightness_ratio = 0.05
saturation_ratio = 0.05
hue_ratio = 0.02

for j in range(10):
    #select hramdomly how many objects will be in an image
    num_objects = random.randint(1, 3)
    # print("number of objects",num_objects)

    # Select random foreground images from the three folders, with replacement
    #fg_categories = random.choices("nao", k=num_objects)

    fg_files_selected = []
    results_path = os.listdir(results_dir)
    for i in (0,num_objects):
        fg_files_selected.append(random.choice(results_path))
    print("seleccion",fg_files_selected)

    # Load the selected foreground images using Pillow -- img[0] is image, img[1] is path
    fg_imgs = []
    images_dir = "raw/"
    for img in fg_files_selected:
        folder = "output/"
        if img[0] == 'N':
          fg_imgs.append([Image.open(folder + img),folder+img])

    # Randomly resize and rotate the foreground images using Pillow's transform module
    # img[0] is image, img[1] is path
    for img in fg_imgs:
        fg_img=img[0]
        if (fg_img.height > 0 and fg_img.width > 0):
          angle = random.randint(-5, 5)
          scale = random.uniform(rescaling_min, rescaling_max)
          fg_img = fg_img.rotate(angle, resample=Image.BICUBIC, expand=True)
          fg_img = fg_img.resize((int(fg_img.width * scale), int(fg_img.height * scale)))
          fg_img = ImageEnhance.Brightness(fg_img).enhance(random.uniform(1.0, 1.5))
          fg_img = ImageEnhance.Contrast(fg_img).enhance(random.uniform(0.9, 1.1))
          fg_img = ImageEnhance.Color(fg_img).enhance(random.uniform(0.7, 1.3))
          fg_img = fg_img.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.0, 0.5)))


        img[0] = fg_img

    # Load the background image using Pillow
    bg_folder = "backgrounds/"
    bg_files = os.listdir(bg_folder)
    bg_file = random.choice(bg_files)
    if bg_file[0] != ".":
      bg_img = Image.open(bg_folder + bg_file)

      # Define the maximum overlap as a percentage
    max_overlap_pct = 10

    # Define an array to keep track of occupied areas
    occupied = np.zeros((bg_img.height, bg_img.width))

    print(fg_imgs)

    for img in fg_imgs:
        fg_img=img[0]

        # Calculate the maximum overlap area
        max_overlap_area = (fg_img.width * fg_img.height)

        seg_img = img[0]
        print(seg_img)

        # Convert the image to a NumPy array
        img_arr = np.array(seg_img)

        # Create a binary mask of the non-transparent pixels
        mask = img_arr[:, :, 3] != 0

        # Convert the mask to a COCO format segmentation
        segmentation = []
        for i in range(mask.shape[0]):
            for j in range(mask.shape[1]):
                if mask[i, j]:
                    segmentation.append(j)
                    segmentation.append(i)
        segmentation = [segmentation]

        #Calculate the area of the segmentation
        area = 0
        for i in range(len(segmentation[0]) // 2):
            x1 = segmentation[0][2 * i]
            y1 = segmentation[0][2 * i + 1]
            x2 = segmentation[0][(2 * i + 2) % len(segmentation[0])]
            y2 = segmentation[0][(2 * i + 3) % len(segmentation[0])]
            area += x1 * y2 - x2 * y1
        area = abs(area) / 2


        # Calculate the maximum allowed position for the top-left corner
        max_x = bg_img.width - fg_img.width
        max_y = bg_img.height - fg_img.height
        area = fg_img.width * fg_img.height

        # Generate a random location until an unoccupied area is found that meets the overlap limit
        total_area = bg_img.width * bg_img.height
        overlap_area = total_area

        while overlap_area / area > max_overlap_pct / 100:
            x = random.randint(0, max_x)
            y = random.randint(0, max_y)

            # Calculate the overlap area
            overlap_area = np.sum(occupied[y:y+fg_img.height, x:x+fg_img.width])

            # Check if the area is unoccupied and the overlap limit is not exceeded
            if (max_overlap_area) >= overlap_area/10:
                break
            if i==10:
                continue

        for i in range(0, len(segmentation[0])):
            if i % 2:
                segmentation[0][i]=int(segmentation[0][i]+y)
            else :
                segmentation[0][i]=int(segmentation[0][i]+x)
        # Update the occupied array
        occupied[y:y+fg_img.height, x:x+fg_img.width] = 1
        bg_img.paste(fg_img, (x, y), fg_img)
        annotations.append({"id": anno_id,"image_id": img_id,"category_id": 1,"bbox": [x, y, fg_img.width, fg_img.height],"segmentation": [],"area": fg_img.height*fg_img.width,"iscrowd": 0})
        anno_id=anno_id+1

    # copy = np.array(bg_img)
    # cv2.imwrite(os.path.join(dataset, file), copy)

    bg_img.save("dataset/"+str(img_id)+".jpg", quality=100)
    images.append({"id": img_id, "file_name": str(img_id)+".jpg","height": bg_img.height,"width": bg_img.width})
    img_id=img_id+1


In [ ]:
#results_dir
#dataset
#bgs

fg_files = os.listdir(results_dir)

seleccion ['NAO14.png', 'NAO9.png']
[[<PIL.Image.Image image mode=RGBA size=437x767 at 0x7B56B87E3CA0>, 'output/NAO14.png'], [<PIL.Image.Image image mode=RGBA size=1x1 at 0x7B56B87E0FD0>, 'output/NAO9.png']]
<PIL.Image.Image image mode=RGBA size=437x767 at 0x7B56B87E3CA0>
<PIL.Image.Image image mode=RGBA size=1x1 at 0x7B56B87E0FD0>
seleccion ['NAO1.png', 'NAO10.png']
[[<PIL.Image.Image image mode=RGBA size=1128x1556 at 0x7B56B87E18D0>, 'output/NAO1.png'], [<PIL.Image.Image image mode=RGBA size=2x2 at 0x7B56B87E21D0>, 'output/NAO10.png']]
<PIL.Image.Image image mode=RGBA size=1128x1556 at 0x7B56B87E18D0>
<PIL.Image.Image image mode=RGBA size=2x2 at 0x7B56B87E21D0>
seleccion ['NAO9.png', 'NAO10.png']
[[<PIL.Image.Image image mode=RGBA size=1x1 at 0x7B56B87E25C0>, 'output/NAO9.png'], [<PIL.Image.Image image mode=RGBA size=4x4 at 0x7B56B87E2F20>, 'output/NAO10.png']]
<PIL.Image.Image image mode=RGBA size=1x1 at 0x7B56B87E25C0>
<PIL.Image.Image image mode=RGBA size=4x4 at 0x7B56B87E2F20>
se

In [ ]:

import random
import json
from PIL import Image, ImageDraw, ImageEnhance, ImageFilter
from pycocotools import mask
from skimage import measure

[[<PIL.Image.Image image mode=RGBA size=4x2 at 0x7B56F1174880>, 'output/NAO15.png'], [<PIL.Image.Image image mode=RGBA size=354x486 at 0x7B56F11772E0>, 'output/NAO6.png']]
<PIL.Image.Image image mode=RGBA size=4x2 at 0x7B56F1174880>
<PIL.Image.Image image mode=RGBA size=354x486 at 0x7B56F11772E0>


In [ ]:
# Define the COCO dictionary
coco_dict = {
    "images": images,
    "annotations": annotations,
    "categories": "NAO"
}

In [ ]:
coco_dict
# Convert the dictionary to a JSON string
json_string = json.dumps(coco_dict)

# Write the JSON string to a file
with open("annotations_val.json", 'w') as f:
    f.write(json_string)

In [ ]:
MY_KEY = "OHK7KhOXnobE9SiVDQhi"

for filename in os.listdir(dataset):
  if filename.endswith(".jpg") or filename.endswith(".png"):
      print(filename)

      # Load Image with PIL
      image = Image.open(filename).convert("RGB")

      # Convert to JPEG Buffer
      buffered = io.BytesIO()
      image.save(buffered, quality=90, format="JPEG")

      # Base 64 Encode
      img_str = base64.b64encode(buffered.getvalue())
      img_str = img_str.decode("ascii")

      # Construct the URL
      upload_url = "".join([
          "https://app.roboflow.com/roborregos/test-vkojw/upload",
          "?api_key=" + MY_KEY,
          "&name=" +str(filename),
          "&split=train"
      ])

      # POST to the API
      r = requests.post(upload_url, data=img_str, headers={
          "Content-Type": "application/x-www-form-urlencoded"
      })

      # Output result
      print(r.json())

In [ ]:
# from ultralytics.data.converter import convert_coco

# newFile = convert_coco(labels_dir='annotations.json')


None
